# Clase 17: Clustering

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

**Profesor: Ignacio Meza**


## Objetivos de la Clase


- Comprender cuál es la utilidad de las técnicas de clustering.
- Analizar diversos tipos de algoritmos de clustering.

## Clustering

Clustering es la tarea que consiste en agrupar observaciones similares en grupos llamados *clusters*. La idea es que los grupos solo contengan información similar.
Es una tarea usual al realizar Análisis Exploratorio de Datos (EDA), ya que permite encontrar de forma automatizada grupos de observaciones similares.

Ya que no es necesario que el dataset esté etiquetado, es una técnica de aprendizaje no-supervisado.

<div align='center'>
    <img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/clustering.png?raw=true' width=800/>
</div>

<div align='center'>
    <span>Ejemplo de Clustering. Fuente: <a href='https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html'>Comparación de Clustering en Scikit-Learn.</a></span>
</div>

## Tipos de Clustering

Existen varias técnicas de clustering, las cuales se pueden clasificar en las siguientes categorías:


|  | **Particional** | **Jerárquico** | **Difuso** |
|---|---|---|---|
| Descripción | Divide los datos en clusters sin traslape, tal que cada dato está en un solo grupo y en ningún otro. | Agrupa ejemplos al ir estableciendo jerarquías entre estos, de tal manera que los datos son organizados como un árbol. | Cada objeto pertenece a cada cluster con un peso de pertenencia entre 0 y 1. |
| Ejemplos | K-Means, DBScan | Aglomerativo, Divisivo | Mixtura de Gaussianas |

## Clustering en `Scikit-learn`

Scikit-learn ofrece una gran gama de algoritmos de clustering para explorar.





<div align='center'>
    <img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/cluster_comparison.png?raw=true' width=800/>
</div>

<div align='center'>
    <span>Ejemplo de los distintos métodos de Clustering ofrecidos por Scikit-Learn. </span>
    <br>
    Fuente: <a href='https://scikit-learn.org/stable/modules/clustering.html'>Clustering en Scikit-Learn.</a>
</div>

---

## Problema de Hoy: 🎸🤘 Caracterización Musical 🎼🎵 

<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/spotify.png?raw=true' width=200/>
</div>

 
    
Los atributos son: 

- `key`: escala de la canción. 0 = C, 1 = C♯/D♭, 2 = D...  [Mas información](https://en.wikipedia.org/wiki/Pitch_class).
- `modo`: 1 si la escala es mayor, 0 si es menor.
- `time_signature`: cuántos pulsos hay en cada compás. (4, 3,...).
- `loudness`: Volumen de la canción (rango -60, 0).


- `acousticness`: Probabilidad de que la canción sea solo acústica.
- `danceability`: Describe que tan bailable es la canción. (rango 0, 1).
- `energy`: Mide que tan energética es una canción (rango 0, 1).
- `instrumentalness`: Probabilidad que la canción contenga voces.
- `liveness`: Probabilidad de que la canción fuese grabada en vivo.
- `speechiness`: Probabilidad de que la canción sea exclusivamente vocal (ejemplo: podcast : 1). 
- `valence`: Sentimiento de la canción (rango 0, 1). 1 -> felicidad, alegria, euforia. 0 -> Tristeza, enojo, depresión.
- `tempo` : Pulsos por minuto de la canción (BPM). 


La variable a predecir es: 

- `genre`: Género de la canción.


**Pregunta**: A simple vista,

- ¿Hay carácterísticas que podrían estas repetidas? (**Irrelevante**)
- ¿hay características que nos dicen mas o menos lo mismo? (**Redundante**)


### Análisis Exploratorio de Datos

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("/Users/imezadelajara/Repositories/MDS7202/recursos/2023-01/17-Clustering/descriptores_musica.csv")
df.head(5)

In [ ]:
df.describe()

In [ ]:
def get_ejemplo(idx):
    """
    Obtiene un ejemplo y lo formatea como columna.
    """
    ejemplo = (
        df.loc[
            idx,
            [
                "danceability",
                "energy",
                "speechiness",
                "acousticness",
                "instrumentalness",
                "valence",
                "name",
                "artist",
                "genre",
            ],
        ]
        .to_frame()
        .reset_index()
    )
    ejemplo.columns = ["Descriptor", "Valor"]
    return ejemplo

In [ ]:
# pueden cambiar el índice de alguno de estos ejemplos para
# mostrar otra canción en la visualización
ejemplo1 = get_ejemplo(102)
ejemplo2 = get_ejemplo(385)
ejemplo3 = get_ejemplo(15)
ejemplo4 = get_ejemplo(484)

ejemplos = [ejemplo1, ejemplo2, ejemplo3, ejemplo4]

#### Spider/Radar Chart

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[
        [{"type": "polar"}, {"type": "polar"}],
        [{"type": "polar"}, {"type": "polar"}],
    ],
    subplot_titles=[
        ejemplo1.loc[6, "Valor"], ejemplo3.loc[6, "Valor"],
        ejemplo2.loc[6, "Valor"], ejemplo4.loc[6, "Valor"],
    ],
)

for i, ejemplo in enumerate(ejemplos):
    fig.add_trace(
        go.Scatterpolar(
            r=ejemplo.loc[0:5, "Valor"],
            theta=ejemplo.loc[0:5, "Descriptor"],
            fill="toself",
            name=f"{ejemplo.loc[6, 'Valor']} - {ejemplo.loc[7, 'Valor']} ({ejemplo.loc[8, 'Valor']})",
        ),
        col=i // 2 + 1,
        row=i % 2 + 1,
    )

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    showlegend=False,
    title="ScatterPolar/Radar/Spider Chart/ Descripción de Ejemplos",
    height=700,
)

fig.show()

#### Correlaciones

In [ ]:
corr = df.loc[
    :,
    [
        "danceability",
        "energy",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "valence",
        "name",
        "artist",
        "genre",
    ],
].corr()

#### Histogramas

In [ ]:
px.histogram(df, x="duration_ms")

In [ ]:
px.histogram(df, x="loudness")

In [ ]:
px.histogram(df, x="tempo")

In [ ]:
dt_to_hists = df.loc[
    :,
    [
        "danceability",
        "energy",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "valence",
        "liveness",
        "genre",
    ],
].melt(id_vars=["genre"], var_name="variable", value_name="valor")

px.histogram(
    dt_to_hists, x="valor", color="variable", facet_col="variable", facet_col_wrap=4
).update_layout(showlegend=False)

### Correlaciones

In [ ]:
px.imshow(corr)

---

## Preparando los Datos

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

Para este clase usaremos los siguientes atributos:

In [ ]:
df_ = df.loc[
    :,
    [
        "danceability",
        "energy",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "valence",
        "liveness",
        "duration_ms",
        "loudness",
        "name",
        "genre",
    ],
]

df_

In [ ]:
df_.describe()

### `MinMaxScaler`

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

ct = ColumnTransformer(
    [("MinMax", MinMaxScaler(), ["duration_ms", "loudness"])], remainder="passthrough"
)


pipe = Pipeline([("Preprocesamiento", ct)])

In [ ]:
features_to_scale = df_.iloc[:, :-2]  # eliminar nombre y género

pipe.fit_transform(features_to_scale)

In [ ]:
# transformamos el resultado de la transformación a un dataframe:

scaled_features = pd.DataFrame(
    pipe.fit_transform(features_to_scale), columns=features_to_scale.columns
)

scaled_features

In [ ]:
scaled_features.describe()

---

## `UMAP` - Proyectamos con UMAP

In [ ]:
from umap import UMAP

proyector = UMAP(random_state=88, n_neighbors=20, min_dist=0.15)

projections = proyector.fit_transform(scaled_features)

In [ ]:
projections.shape

In [ ]:
# este dataframe lo estaremos usando para graficar de aquí en adelante
fig_df = pd.concat(
    [
        df.loc[:, ["name", "artist", "genre"]],
        scaled_features,
        pd.DataFrame(projections, columns=["x", "y"]),
    ],
    axis=1,
)

fig_df

In [ ]:
def get_scatter(fig_df, color_col):
    fig = px.scatter(
        fig_df,
        x="x",
        y="y",
        color=color_col,
        hover_name=df["artist"] + " - " + df["name"],
        labels={"genre": "Género Musical"},
        hover_data=[
            "danceability",
            "energy",
            "speechiness",
            "acousticness",
            "instrumentalness",
            "valence",
        ],
        range_x=(fig_df["x"].min() - 1, fig_df["x"].max() + 1),
        range_y=(fig_df["y"].min() - 1, fig_df["y"].max() + 1),
    )
    return fig


get_scatter(fig_df, "genre")

---

**Aquí empezamos clustering**

### K-Means

Técnica de clustering de tipo particional.

Encuentra centros de clusters que minimizan la suma de distancias entre los datos y el centro de cada cluster.

Algoritmo:

---


    Seleccionar K centroides iniciales.
    repite: 
        Asigna todos los puntos a sus centros más cercanos.
        Recomputa los centros de cada cluster.
    hasta que los centros no cambien.

---
*Hasta que los centros no cambian* equivale a minimizar la suma de errores cuadrados SSE:

$$SSE = \sum_{i=1}^{K} \sum_{x\in C_i} d(c_i, x)^2 $$

Comunmente la función de distancia $d$ es la distancia euclideana $d(x,y) = \sqrt{\sum_{i}(x_i, y_i)^2}$ .

<div align='center'>

<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/kmeans_example.png?raw=true' width=500/>
</div>

<div align='center'>
    Fuente: <a href='https://www.jparzival.com/blog/como-funciona-k-means/'>
¿Cómo funciona K-Means?
 en jparzival.com
</div>


> **Pregunta**: ¿Qué sucede si se elige un mal ejemplo inicial?

In [ ]:
# Random State permite controlar la aleatoridad.
# Es decir, permite generar los mismos números aleatorios en distintas ejecuciones.
RANDOM_STATE = 99

In [ ]:
from sklearn.cluster import KMeans

# El número de clusters es parámetro. En este caso, es 2.
kmeans = KMeans(n_clusters=2, random_state=RANDOM_STATE).fit(scaled_features)

labels = kmeans.labels_

Podemos acceder a los centroides calculados.

In [ ]:
scaled_features.iloc[0]

In [ ]:
# Clusters calculados por cada observación de entrenamiento
labels

In [ ]:
# Centroides calculados.
kmeans.cluster_centers_

In [ ]:
# También podemos acceder a la suma de errores cuadráticos (llamada inercia en scikit)
kmeans.inertia_

In [ ]:
fig_df["kmeans_labels_2"] = kmeans.labels_
fig = get_scatter(fig_df, "kmeans_labels_2")

# Transformamos los centro que vimos anteriormentes a la proyección 2d.
projected_centers = proyector.transform(kmeans.cluster_centers_)

fig.add_trace(
    go.Scatter(
        x=projected_centers[:, 0],
        y=projected_centers[:, 1],
        mode="markers",
        name="Centros",
        marker_size=12,
        marker_color="LightSlateGray",
    )
)

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(scaled_features)
labels = kmeans.labels_


fig_df["kmeans_labels_5"] = kmeans.labels_
fig = get_scatter(fig_df, "kmeans_labels_5")

In [ ]:
# Transformamos los centro que vimos anteriormentes a la proyección 2d.
centros_pryectados = proyector.transform(kmeans.cluster_centers_)

fig = get_scatter(fig_df, "kmeans_labels_5")
fig.add_trace(
    go.Scatter(
        x=centros_pryectados[:, 0],
        y=centros_pryectados[:, 1],
        mode="markers",
        name="Centros",
        marker_size=12,
        marker_color="LightSlateGray",
    )
)

> **Pregunta:** ¿Qué pasaría si ejecutamos el clustering con los datos no escalados?

> **Pregunta:** ¿Cómo identificamos la cantidad de cluster óptimos?


### Método del Codo



![Método del Codo](https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/elbow.png?raw=true)

In [ ]:
intertias = [
    [i, KMeans(n_clusters=i, random_state=0).fit(scaled_features).inertia_]
    for i in range(2, 20)
]

intertias = pd.DataFrame(intertias, columns=["n° clusters", "inertia"])
intertias.head(10)

In [ ]:
px.line(
    intertias,
    x="n° clusters",
    y="inertia",
    title="Método del Codo con K-Means",
    height=600,
)

**Alternativa: Coeficiente de Silohuette**


El valor de la silueta es una medida de cuán similar es un objeto respecto a su propio cluster en comparación con los otros.  


Por cada elemento se calcula: 

$$s = \frac{b-a}{\max(a,b)}$$

donde: 

- $a$ es el promedio de las distancias entre el elemento y todos los otros elementos del cluster al cual el elemento analizado fue etiquetado.
- $b$ es la distancia entre el elemento y el centro del cluster más cercano.

La silueta va de -1 a +1 donde 
- un valor cercano a 1: indica que el objeto está bien clusterizado
- cercano a 0: indica que el elemento está entre dos clusters
- cercano a $-1$: indica que el elemento está mal asignado. 

Si la mayoría de los objetos tienen un valor alto, entonces podemos decir que los elementos del cluster están bien asignados.

Por último, el coeficiente de silueta se calcula al promediar todos los coeficientes individuales.

In [ ]:
from sklearn.metrics import silhouette_score

scores = [
    [
        i,
        silhouette_score(
            scaled_features,
            KMeans(n_clusters=i, random_state=99).fit(scaled_features).labels_,
        ),
    ]
    for i in range(2, 20)
]

scores = pd.DataFrame(scores, columns=["n° clusters", "silhouette_score"])
scores.head(10)

In [ ]:
px.line(scores, x="n° clusters", y="silhouette_score")

**Más opciones:**

https://medium.com/@haataa/how-to-measure-clustering-performances-when-there-are-no-ground-truth-db027e9a871c

---


## `DBSCAN`
	

Algoritmo de clustering basado en densidad. Ideal para buscar outliers.

- Densidad: Número de puntos en un círculo.
- Idea: Regiones densas representan clusters.

Parámetros: 

- `Eps`: radio de los círculos
- `MinPts`: número mínimo de puntos de una región.


- Es comunmente resistente al ruido.
- Problemas en regiones con distintas densidades.


In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.4, min_samples=3)
dbscan.fit(scaled_features)

fig_df["dbscan"] = pd.Series(dbscan.labels_, dtype=str)

get_scatter(fig_df, "dbscan")

### Visualizando el parámetro eps

In [ ]:
import numpy as np

epss = np.arange(0.1, 1, 0.2)
epss

In [ ]:
clustering = [
    DBSCAN(eps=eps, min_samples=2).fit_predict(scaled_features) for eps in epss
]

dbscan_labels = pd.DataFrame(np.array(clustering)).T
dbscan_labels.columns = epss

dbscan_labels["x"] = projections[:, 0]
dbscan_labels["y"] = projections[:, 1]

dbscan_labels = dbscan_labels.melt(
    id_vars=["x", "y"], var_name="eps", value_name="label"
)
dbscan_labels["label"] = dbscan_labels["label"].astype(str)
dbscan_labels.sample(10)

In [ ]:
fig = px.scatter(
    dbscan_labels,
    x="x",
    y="y",
    facet_row="eps",
    color="label",
    # range_x=(projections[:, 0].min() - 1, projections[:, 1].max() + 1),
    # range_y=(projections[:, 0].min() - 1, projections[:, 1].max() + 1),
    height=1600,
)
fig.show()

---

## Clustering Jerárquico
</br>
<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/clustering_jerarquico.png?raw=true' width=400 />
</div>

> **Pregunta**: ¿Cómo los agrupamos?

Requieren que exista una **definición de distancia** entre los elementos que se desean agrupar.

<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/matriz_distancias.png?raw=true' width=800 />
</div>


### Tipos

#### Aglomerativo

- Empezar con cada punto como cluster individual•
- En cada paso, mezclar el par de clusters más cercano hasta que quede sólo un cluster (o k clusters) 

#### Divisivo

- Empezar con un cluster que contenga todos los puntos
- En cada paso, dividir un cluster en dos hasta que todo cluster contenga un solo punto (o haya k clusters)




### Algortimo básico Aglomerativo


---

    Partimos con que cada punto es cluster por separado.
    Calculamos la matriz de distancias.
    Repetimos :
        Unimos los puntos/clusters (usando la matriz de distancias) en un solo cluster según algún criterio/enlace.
    hasta que ya no podamos unir nada más.
---


### Tipos de Enlaces entre Clusters


El enlace determina como se irán uniendo los distintos clusters que se irán generando. Existen varias opciones: 

- **Máx - Enlace Completo** (`complete`): Va uniendo puntos/clusters mas lejanos.


- **Min - Enlace Simple** (`simple`): Va uniendo puntos/clusters mas cercanos.


- **Promedio entre grupos** (`average`): Va uniendo según el promedio de la distancia de todos contra todos


- **Ward (`ward`)**: Va uniendo al minimizar la suma de las diferencias cuadradas entre cluster. Muy similar a lo que hace K-Means.


La opción elegida puede provocar variaciones gigantezcas entre los clusters producidos.

In [ ]:
import numpy as np
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import average, complete, dendrogram, single, ward
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import load_iris

#### Máx - Enlace Completo (`complete`)
    
<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/agglomerative_max.png?raw=true' width=400 />
</div>
    
$$\max\{(d(x,y): x \in A, y \in B)\}$$  
    
    
- Poco suceptible a outliers.
- Tiende a quebrar clusters grandes. 
- Tiende a formar clusters esféricos.

In [ ]:
sample = scaled_features.sample(20, random_state=RANDOM_STATE)
sample

In [ ]:
# plot the top three levels of the dendrogram
ff.create_dendrogram(
    sample,
    labels=(
        df.loc[sample.index, "artist"] + " - " + df.loc[sample.index, "name"]
    ).values,
    linkagefun=complete,
    orientation="left",
    color_threshold=1.5,
).update_layout(width=1000, height=800)

In [ ]:
model = AgglomerativeClustering(n_clusters=5, linkage="complete")
labels = model.fit_predict(scaled_features)

fig_df["ag_complete"] = pd.Series(model.labels_, dtype=str)

get_scatter(fig_df, "ag_complete")

#### Mín - Enlace Simple (`single`)

<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/agglomerative_min.png?raw=true' width=400 />
</div>

$$\min\{(d(x,y): x \in A, y \in B)\}$$  


- Puede manejar formas no elípticas
- Tiende a romper clusters.
- Sensible a ruido y outliers.




In [ ]:
# plot the top three levels of the dendrogram
ff.create_dendrogram(
    sample,
    labels=(
        df.loc[sample.index, "artist"] + " - " + df.loc[sample.index, "name"]
    ).values,
    linkagefun=single,
    orientation="left",
    color_threshold=0.4,
).update_layout(width=1000, height=800)

In [ ]:
model = AgglomerativeClustering(n_clusters=5, linkage="single")
labels = model.fit_predict(scaled_features)


fig_df["ag_single"] = pd.Series(model.labels_, dtype=str)
get_scatter(fig_df, "ag_single")

#### Promedio (`average`)

<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/agglomerative_mean.png?raw=true' width=400 />
</div>


- Compromiso entre min y max.
- Menos suceptible a ruidos y outliers.
- Sesgado a clusters esféricos

In [ ]:
# plot the top three levels of the dendrogram
ff.create_dendrogram(
    sample,
    labels=(
        df.loc[sample.index, "artist"] + " - " + df.loc[sample.index, "name"]
    ).values,
    linkagefun=average,
    orientation="left",
    color_threshold=0.8,
).update_layout(width=1000, height=800)

In [ ]:
model = AgglomerativeClustering(n_clusters=5, linkage="average")
labels = model.fit_predict(scaled_features)

fig_df["ag_average"] = pd.Series(model.labels_, dtype=str)
get_scatter(fig_df, "ag_average")

#### Ward (`ward`)

$$\frac{1}{|A|\cdot|B|} \sum_{x \in A} \sum_{x \in B} d(x,y)$$
    

Busca minimizar el incremento del SSE cuando se mezclan dos clusters


In [ ]:
# plot the top three levels of the dendrogram
ff.create_dendrogram(
    sample,
    labels=(
        df.loc[sample.index, "artist"] + " - " + df.loc[sample.index, "name"]
    ).values,
    linkagefun=ward,
    orientation="left",
    color_threshold=0.8,
).update_layout(width=1000, height=800)

In [ ]:
model = AgglomerativeClustering(n_clusters=5, linkage="ward")
labels = model.fit_predict(scaled_features)

fig_df["ag_ward"] = pd.Series(model.labels_, dtype=str)
get_scatter(fig_df, "ag_ward")

### Resumen clustering jerárquicos

Ventajas: 
    
- No hay que suponer a priori el número de clases.
- Podemos escoger el número de clusters cortando el dendograma. 


Desventajas:

- No hay función objetivo por minimizar.
- Diferentes esquemas presentan diferentes problemas.
- Memoria $O (n^2)$, Tiempo $O (N^2\log(N)$